In [14]:
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from joblib import Parallel, delayed
import time

# --- 1. STRUTTURA DATI UNION-FIND ---
class UnionFind:
    def __init__(self, size):
        self.parent = np.arange(size)
        self.size = np.ones(size, dtype=int)
        self.max_size = 0

    def find(self, i):
        # Compressione del cammino iterativa (molto più sicura ed efficiente)
        root = i
        while self.parent[root] != root:
            root = self.parent[root]
        
        curr = i
        while curr != root:
            nxt = self.parent[curr]
            self.parent[curr] = root
            curr = nxt
        return root

    def union(self, i, j):
        root_i = self.find(i)
        root_j = self.find(j)
        
        if root_i != root_j:
            # Unione per dimensione: attacchiamo il più piccolo al più grande
            if self.size[root_i] < self.size[root_j]:
                root_i, root_j = root_j, root_i
                
            self.parent[root_j] = root_i
            self.size[root_i] += self.size[root_j]
            
            # Aggiorniamo la dimensione massima globale se necessario
            if self.size[root_i] > self.max_size:
                self.max_size = self.size[root_i]

# --- 2. FUNZIONE PER IL SINGOLO RUN (Newman-Ziff) ---
def simulate_single_run_uf(N, k, f_sim):
    """Simula la percolazione inversa ricostruendo la rete dal basso."""
    p_prob = k / (N - 1)
    
    # Generiamo l'intera topologia una volta sola
    G = ig.Graph.Erdos_Renyi(n=N, p=p_prob, directed=False)
    adj_list = G.get_adjlist()  # Estrazione ultraveloce delle adiacenze
    
    # Ordine di "accensione" dei nodi (inverso della rimozione)
    nodi_shuffled = np.arange(N)
    np.random.shuffle(nodi_shuffled)
    
    uf = UnionFind(N)
    is_active = np.zeros(N, dtype=bool)
    
    # Array per salvare la storia della componente gigante passo dopo passo
    max_size_history = np.zeros(N + 1, dtype=int)
    
    n_active = 0
    for node in nodi_shuffled:
        is_active[node] = True
        n_active += 1
        
        # Un singolo nodo appena acceso forma un cluster di dimensione 1
        if uf.max_size == 0:
            uf.max_size = 1 
            
        # Controlliamo i vicini originali: se sono attivi, li uniamo
        for neighbor in adj_list[node]:
            if is_active[neighbor]:
                uf.union(node, neighbor)
                
        # Registriamo la grandezza della GC in questo istante
        max_size_history[n_active] = uf.max_size

    # Ora leggiamo i risultati calcolati in corrispondenza dei nostri f_sim
    run_results = np.zeros(len(f_sim))
    for j, f in enumerate(f_sim):
        N_keep = int(N * (1 - f))
        # Salviamo S normalizzato sul numero totale N
        run_results[j] = max_size_history[N_keep] / N
        
    return run_results

def build_adaptive_f_grid(fc, f_min=0.0, f_max=0.99, dense_half_width=0.12,
                          n_left=10, n_center=30, n_right=10):
    """Costruisce una griglia f non uniforme: densa vicino a fc e rada altrove."""
    left_edge = max(f_min, fc - dense_half_width)
    right_edge = min(f_max, fc + dense_half_width)

    parts = []

    if left_edge > f_min:
        parts.append(np.linspace(f_min, left_edge, n_left, endpoint=False))
    else:
        parts.append(np.array([f_min]))

    parts.append(np.linspace(left_edge, right_edge, n_center, endpoint=False))

    if right_edge < f_max:
        parts.append(np.linspace(right_edge, f_max, n_right))
    else:
        parts.append(np.array([f_max]))

    f_grid = np.unique(np.concatenate(parts))
    f_grid = f_grid[(f_grid >= f_min) & (f_grid <= f_max)]
    return f_grid

# --- 3. PARAMETRI E FIT ---
N = 100000
M = 20
k_list = np.arange(2, 12)

# Parametri della griglia adattiva in f (meno punti totali, più risoluzione vicino a fc)
DENSE_HALF_WIDTH = 0.12
N_F_LEFT = 10
N_F_CENTER = 30
N_F_RIGHT = 10

def linear_scaling(x, A, b): return A * (b - x)
def teoria_fc(k, c): return 1 - (c / k)

S_norm_tutti, S_std_tutti = {}, {}
f_sim_tutti = {}
fc_misurati, fc_err = [], []

# --- 4. MOTORE PARALLELO ---
print(f"Avvio simulazione Union-Find parallela (N={N}, M={M})...")
start_total = time.time()

for k in k_list:
    t0 = time.time()
    fc_atteso = 1 - (1 / k)
    f_sim = build_adaptive_f_grid(
        fc_atteso,
        dense_half_width=DENSE_HALF_WIDTH,
        n_left=N_F_LEFT,
        n_center=N_F_CENTER,
        n_right=N_F_RIGHT
    )
    
    # Lanciamo gli M grafi sui core disponibili
    risultati = Parallel(n_jobs=-1)(
        delayed(simulate_single_run_uf)(N, k, f_sim) for _ in range(M)
    )
    S_results = np.array(risultati)

    # Normalizzazione S(f) / S(0)
    S_norm_runs = np.divide(
        S_results, S_results[:, [0]],
        out=np.zeros_like(S_results), where=S_results[:, [0]] != 0
    )
    S_norm = np.mean(S_norm_runs, axis=0)
    S_std = np.std(S_norm_runs, axis=0, ddof=1) if M > 1 else np.zeros_like(S_norm)
    
    S_norm_tutti[k] = S_norm
    S_std_tutti[k] = S_std
    f_sim_tutti[k] = f_sim
    
    # Fit Dinamico
    mask = (f_sim >= fc_atteso - 0.15) & (f_sim <= fc_atteso - 0.03) & (S_norm < 0.8)
    
    try:
        popt, pcov = curve_fit(linear_scaling, f_sim[mask], S_norm[mask], p0=[2.0, fc_atteso])
        fc_misurato, err = popt[1], np.sqrt(pcov[1, 1])
    except:
        fc_misurato, err = np.nan, np.nan
        
    fc_misurati.append(fc_misurato)
    fc_err.append(err)
    print(f"k={k:2d} | N_f={len(f_sim):2d} | fc_teo={fc_atteso:.3f} | fc_mis={fc_misurato:.3f} | Tempo: {time.time()-t0:.1f} s")

fc_misurati, fc_err = np.array(fc_misurati), np.array(fc_err)
print(f"\nCompletato in {time.time() - start_total:.2f} s totali!")

Avvio simulazione Union-Find parallela (N=100000, M=20)...
k= 2 | N_f=50 | fc_teo=0.500 | fc_mis=0.503 | Tempo: 11.0 s
k= 3 | N_f=50 | fc_teo=0.667 | fc_mis=0.672 | Tempo: 14.0 s
k= 4 | N_f=50 | fc_teo=0.750 | fc_mis=0.754 | Tempo: 16.8 s
k= 5 | N_f=50 | fc_teo=0.800 | fc_mis=0.805 | Tempo: 20.4 s
k= 6 | N_f=50 | fc_teo=0.833 | fc_mis=0.840 | Tempo: 23.0 s
k= 7 | N_f=50 | fc_teo=0.857 | fc_mis=0.865 | Tempo: 25.1 s
k= 8 | N_f=41 | fc_teo=0.875 | fc_mis=0.885 | Tempo: 28.9 s
k= 9 | N_f=41 | fc_teo=0.889 | fc_mis=0.898 | Tempo: 31.2 s
k=10 | N_f=41 | fc_teo=0.900 | fc_mis=0.911 | Tempo: 34.9 s
k=11 | N_f=41 | fc_teo=0.909 | fc_mis=0.920 | Tempo: 36.4 s

Completato in 241.76 s totali!


In [6]:
import numpy as np
# --- 5. GRAFICI ---

# Stili globali (come in phase_tr.ipynb)
TITLE_FS = 20
AXIS_FS = 18
TICK_FS = 15
LEGEND_FS = 14

# Fit finale su f_c(k) (usato nel secondo grafico)
valid_mask = np.isfinite(fc_misurati) & np.isfinite(fc_err) & (fc_err > 0)
k_valid = k_list[valid_mask]
fc_valid = fc_misurati[valid_mask]
err_valid = fc_err[valid_mask]

if len(k_valid) >= 2:
    popt_k, pcov_k = curve_fit(teoria_fc, k_valid, fc_valid, sigma=err_valid, absolute_sigma=True)
    c_opt = popt_k[0]
    c_err = np.sqrt(pcov_k[0, 0])

    residui = fc_valid - teoria_fc(k_valid, c_opt)
    chi2 = np.sum((residui / err_valid) ** 2)
    dof = len(k_valid) - 1
    chi2_ridotto = chi2 / dof if dof > 0 else np.nan
else:
    c_opt = np.nan
    c_err = np.nan
    chi2 = np.nan
    dof = 0
    chi2_ridotto = np.nan

# --- Immagine 1: curve S(f)/S(0) ---
fig1, ax1 = plt.subplots(figsize=(8, 6))
colormap = plt.cm.viridis(np.linspace(0, 1, len(k_list)))

for idx, k in enumerate(k_list):
    ax1.errorbar(
        f_sim_tutti[k],
        S_norm_tutti[k],
        yerr=S_std_tutti[k],
        fmt='-o',
        markersize=2.5,
        linewidth=1.0,
        elinewidth=0.6,
        capsize=1.5,
        color=colormap[idx],
        alpha=0.9,
        label=f'$\\langle k \\rangle = {k}$'
    )

ax1.set_xlabel('$f$', fontsize=AXIS_FS)
ax1.set_ylabel('$S(f) / S(0)$', fontsize=AXIS_FS)
ax1.set_title('Percolazione inversa', fontsize=TITLE_FS)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1.05)
ax1.legend(loc='upper right', fontsize=LEGEND_FS)
ax1.tick_params(axis='x', which='both', labelbottom=True, labelsize=TICK_FS)
ax1.tick_params(axis='y', which='both', labelleft=True, labelsize=TICK_FS)
ax1.grid(True, alpha=0.3)
fig1.tight_layout()
plt.show()

# --- Immagine 2: soglia critica f_c vs k ---
fig2, ax2 = plt.subplots(figsize=(8, 6))
ax2.errorbar(k_list, fc_misurati, yerr=fc_err, fmt='o', markersize=6, color='darkred', label=f'Dati ($N={N}, M={M}$)', capsize=4)

k_smooth = np.linspace(k_list[0], k_list[-1], 100)
if np.isfinite(c_opt):
    ax2.plot(
        k_smooth, teoria_fc(k_smooth, c_opt), 'r--', lw=2,
        label=f'Fit: $\\alpha = {c_opt:.3f} \\pm {c_err:.3f}$ | $\\tilde{{\\chi}}^2 = {chi2_ridotto:.2f}$'
    )
ax2.plot(k_smooth, 1 - 1/k_smooth, 'k-', lw=4, alpha=0.3, label='Teoria ($N \\to \\infty$)')

ax2.set_xlabel('$\\langle k \\rangle$', fontsize=AXIS_FS)
ax2.set_ylabel('$f_c$', fontsize=AXIS_FS)
ax2.set_title('Soglia di Percolazione', fontsize=TITLE_FS)
ax2.legend(loc='lower right', fontsize=LEGEND_FS+2)
ax2.tick_params(axis='x', which='both', labelbottom=True, labelsize=TICK_FS)
ax2.tick_params(axis='y', which='both', labelleft=True, labelsize=TICK_FS)
ax2.grid(True, alpha=0.3)

print('Risultato secondo fit (f_c vs k):')
print(f'  c = {c_opt:.6f} ± {c_err:.6f}')
print(f'  chi^2 = {chi2:.4f}')
print(f'  dof = {dof}')
print(f'  chi^2 ridotto = {chi2_ridotto:.4f}')

fig2.tight_layout()
plt.show()

NameError: name 'fc_misurati' is not defined